First, I should module-ize the definition of the LST grid, and maybe save it out somewhere as additional information about the LST-"sliced" data, since it's a global thing, not a specific to a single file.

I'm also not 100% sure that using the definition of LST in the files, we actually get the same sky above us night-to-night.  That is, for the definition of local sidereal time, is "the" RA that's overhead the RA current epoch?  If so, that will move from night to night.  I think you want function that will give you, for a grid of ICRS coordinates, what time those coordinates appear overhead on a given date.  I'm _sure_ there's not an astropy function for that, unless it is `sidereal_time`.

In [36]:
from astropy import units as u
import numpy as np
from glob import glob
import os
from pyuvdata import UVData
from astropy.coordinates import Angle
import ja

In [2]:
timer = ja.Timer()

In [24]:
# Set up LST grid; in radians
# A trifle worried about numerical precision, but let's plunge ahead

# Files will be this long
lst_bin_size = 6.*u.min
n_lst_bins = int((24.*u.hr/lst_bin_size).to(u.dimensionless_unscaled).value)
lst_edges = np.linspace(0, 2.*np.pi, n_lst_bins+1, endpoint=True)
lst_start = lst_edges[0:-1]
lst_end = lst_edges[1:]
lst_mid = (lst_start + lst_end)/2.

assert len(lst_start) == n_lst_bins
assert len(lst_end) == n_lst_bins
assert np.isclose((lst_end-lst_start)*24.*60./(2.*np.pi), lst_bin_size.value).sum() == n_lst_bins

# Need to specify the time sampling within a "bin"
lst_time_sample = 10*u.s # This is actually every 10 sidereal seconds
n_samples = int((lst_bin_size/lst_time_sample).to(u.dimensionless_unscaled).value)
lst_sampling = ((np.arange(n_samples)*lst_time_sample)/(24*u.hr)).to(u.dimensionless_unscaled).value*2.*np.pi

In [25]:
def jd_at_lst(uvdata, lst0):
    """ Given a UVData object, find the JD corresponding to the requested LST.  
    Uses simple linear interpolation. """
    jd = np.unique(uvdata.time_array)
    lst = np.unique(uvdata.lst_array)
    
    return np.interp(lst0, lst, jd)

In [26]:
jds = [str(jd) for jd in np.arange(2457548, 2457555)]
lstpath = '/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/LSTSlice/'

In [27]:
lstfiles = {}
for jd in jds:
    lstfiles[jd] = np.sort(glob(lstpath+jd+'/lst*.uvh5'))
    print(len(lstfiles[jd]))

116
119
119
120
119
119
120


In [28]:
for jd in jds:
    print(jd, lstfiles[jd][0].split('/')[-1].split('.')[2], lstfiles[jd][-1].split('/')[-1].split('.')[2])

2457548 11h00m00s 22h30m00s
2457549 10h42m00s 22h30m00s
2457550 10h48m00s 22h36m00s
2457551 10h48m00s 22h42m00s
2457552 10h54m00s 22h42m00s
2457553 11h00m00s 22h48m00s
2457554 11h00m00s 22h54m00s


In [29]:
#for jd in jds:
#    os.system('mkdir '+lstpath+jd+'/trimmed')

In [30]:
jdtimer = ja.Timer()
filetimer = ja.Timer()

In [40]:
for jd in jds[1:]:
    jdtimer.start()
    for lstfile in lstfiles[jd]:
        
        filetimer.start()
        
        dirname, filename = os.path.split(lstfile)
        basename, ext = os.path.splitext(filename)
        
        lst_start_str = basename.split('.')[2]
        
        print(lst_start_str)
        
        #uvd = UVData()
        #uvd.read(lstfile)
        
        #lst0 = uvd.lst_array.min()
        #lst0hr = np.round(lst0*24/(2.*np.pi),decimals=2)
        #lst_bin = np.argmin(np.abs(lst0-lst_start))
        #print(lst0, lst_start[lst_bin], lst0-lst_start[lst_bin], lst0hr)
        
        lst0hr = Angle(lst_start_str).hour
        
        if (lst0hr >= 11.0) and (lst0hr <= 22.5):
            
            pass
            #jd_phase = jd_at_lst(uvd, lst_mid[lst_bin])
            #print('Phasing',filename,'to',jd_phase)
            #uvd.phase_to_time(jd_phase)
            
            #uvfitsfile = basename+'.uvfits'
            #print('Writing',uvfitsfile)
            #uvd.write_uvfits(os.path.join(dirname,uvfitsfile), spoof_nonessential=True)
            
        else:
            print('Moving',filename,'to trimmed')
            os.system('mv '+lstfile+' '+os.path.join(dirname,'trimmed/'))
            
        filetimer.stop('File took')
    jdtimer.stop('Day took')

10h42m00s
Moving lst.2457549.10h42m00s.10h47m50s.uvcRP.drift.uvh5 to trimmed
File took 0.21282577514648438 sec
10h48m00s
Moving lst.2457549.10h48m00s.10h53m50s.uvcRP.drift.uvh5 to trimmed
File took 0.0625462532043457 sec
10h54m00s
Moving lst.2457549.10h54m00s.10h59m50s.uvcRP.drift.uvh5 to trimmed
File took 0.058112382888793945 sec
11h00m00s
File took 0.0014662742614746094 sec
11h06m00s
File took 0.0005631446838378906 sec
11h12m00s
File took 0.0005218982696533203 sec
11h18m00s
File took 0.0005071163177490234 sec
11h24m00s
File took 0.0005233287811279297 sec
11h30m00s
File took 0.0004968643188476562 sec
11h36m00s
File took 0.00052642822265625 sec
11h42m00s
File took 0.0005412101745605469 sec
11h48m00s
File took 0.0004856586456298828 sec
11h54m00s
File took 0.00049591064453125 sec
12h00m00s
File took 0.0004787445068359375 sec
12h06m00s
File took 0.0004611015319824219 sec
12h12m00s
File took 0.0004744529724121094 sec
12h18m00s
File took 0.0004649162292480469 sec
12h24m00s
File took 0.00047

File took 0.000301361083984375 sec
12h54m00s
File took 0.0002334117889404297 sec
13h00m00s
File took 0.0002071857452392578 sec
13h06m00s
File took 0.0002396106719970703 sec
13h12m00s
File took 0.00022459030151367188 sec
13h18m00s
File took 0.0001976490020751953 sec
13h24m00s
File took 0.00019741058349609375 sec
13h30m00s
File took 0.00024628639221191406 sec
13h36m00s
File took 0.00020837783813476562 sec
13h42m00s
File took 0.000194549560546875 sec
13h48m00s
File took 0.00019693374633789062 sec
13h54m00s
File took 0.00021004676818847656 sec
14h00m00s
File took 0.00019741058349609375 sec
14h06m00s
File took 0.0001926422119140625 sec
14h12m00s
File took 0.00019311904907226562 sec
14h18m00s
File took 0.00021886825561523438 sec
14h24m00s
File took 0.00021266937255859375 sec
14h30m00s
File took 0.0001919269561767578 sec
14h36m00s
File took 0.00019121170043945312 sec
14h42m00s
File took 0.00022411346435546875 sec
14h48m00s
File took 0.00019502639770507812 sec
14h54m00s
File took 0.00019454956

File took 0.00025773048400878906 sec
17h30m00s
File took 0.00031113624572753906 sec
17h36m00s
File took 0.0003228187561035156 sec
17h42m00s
File took 0.00029277801513671875 sec
17h48m00s
File took 0.00029468536376953125 sec
17h54m00s
File took 0.00029468536376953125 sec
18h00m00s
File took 0.00032210350036621094 sec
18h06m00s
File took 0.0003190040588378906 sec
18h12m00s
File took 0.0002918243408203125 sec
18h18m00s
File took 0.00029587745666503906 sec
18h24m00s
File took 0.0002911090850830078 sec
18h30m00s
File took 0.0002932548522949219 sec
18h36m00s
File took 0.00030803680419921875 sec
18h42m00s
File took 0.00028896331787109375 sec
18h48m00s
File took 0.00031566619873046875 sec
18h54m00s
File took 0.00028896331787109375 sec
19h00m00s
File took 0.0002930164337158203 sec
19h06m00s
File took 0.00029468536376953125 sec
19h12m00s
File took 0.0003001689910888672 sec
19h18m00s
File took 0.00029850006103515625 sec
19h24m00s
File took 0.0002925395965576172 sec
19h30m00s
File took 0.000291824

In [39]:
basename.split('.')[2]

'10h42m00s'

In [ ]:
lstfiles = {}
for jd in jds:
    lstfiles[jd] = np.sort(glob(lstpath+jd+'/lst*.uvh5'))
    print(len(lstfiles[jd]))
for jd in jds:
    print(jd, lstfiles[jd][0].split('/')[-1].split('.')[2], lstfiles[jd][-1].split('/')[-1].split('.')[2])